# ShrimpSimulator
This simulator simulates the Shrimp model.

## Relevant Equations 

### Newton-Euler (equations 2.2 and 2.3)
$$\mathbf{\dot{v}^I_F} = (\mathbf{f_{aero}} + m \mathbf{g} - m \mathbf{\omega^{FI} }\times \mathbf{v^I_F})$$

$$\mathbf{\dot{\omega}^{FI}} = \mathbf{I}^{-1}(\mathbf{\tau_{aero}} - \mathbf{\omega^{FI} }\times \mathbf{I\omega^{BI}})$$

* $v_F^I$ is velocity of the center of mass of flyer frame in inertial/world frame
* $\mathbf{\omega^{FI} } = [p\ q\ r]^T$ is the angular velocity of flyer frame w.r.t. inertial
* $\mathbf{\omega^{BI} }$ is the angular velocity of body frame wrt inertial (same as flyer but with rotation about z axis).


### Blade Equations (equations 3.8)
$$[v_E^I]^E = \mathbf{R^E_F}[v^I_F]^F + \mathbf{R^E_F}[\omega^{SF}]^F \times \mathbf{R^E_F} [S_{EF}]^F + \mathbf{R^E_F}[\nu]^F$$
$$\gamma = arctan(\frac{[v_E^I]^E [\hat{z}]^E}{[v_E^I]^P [\hat{x}]^E})$$


* $[\nu]^F = [0\ 0\ \nu]^T$ is induced inflow velocity
* $\nu$ is the induced inflow velocity (Equation 3.1, thesis p.52)
* $[v_E^I]^E$ is the relative wing in the propeller frame.


### Euler Angles Convention
For euler angles, we are using the `scipy` library of `spatial.transform` with `Rotation`. https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html#scipy.spatial.transform.Rotation

This uses the capital notation `ZXY` to denote intrinsic rotations, which we will use such that the final rotation is about the rotational axis of the vehicle for readibility. This is such that the first rotation is about the Z axis, or corresponds to the yaw.
This matches the MEAM 620 project 1 convention.
Additionally, the euler angles are $\phi, \theta, \psi$.

Please use the `euler2Rotm` and `rotm2Euler` functions for converting between rotation matricies and euler angle vectors 

## Code parameters

* camelCase unless stated otherwise
* Underscores for frames or for subscripts


## Notes
Flyer frame means that it is *not* in the frame of the propellor with the extra spin on the propellor. This is 
only relevant for the shaft propellor.

For the body drag plates, this is all in the body frame (no extra spin) so the euler angular rates are still correct, as well as the euler angles. 

Notation 
* For angular velocities: $\omega$ is a 1x3 vector, while $[\omega]$ is the tensor. We will try to have $[]$ always denote a tensor. 
* For Rotation Matricies: ${}^A R_B$ denotes a rotation from $B$ to $A$, e.g. $p_A = {}^AR_B p_B$
* For position/velocity vectors: ${}^c v_a^b$ denotes a vector from $a$ to $b$ in the $c$ frame.


TODO: read through the math extra carefully again. 
Be explicit with the notation and work through each sub example - write an example for
that one and the next one.

In [2]:
%matplotlib inline
import numpy as np 
from scipy.spatial.transform import Rotation as R

In [3]:
def rotm2Euler(x):
    """Rotation matrix to Euler Angles ZXY intrinsic vector for Shrimp"""
    r = R.from_dcm(x)
    return r.as_euler('ZXY')
    
def euler2Rotm(x):
    """Euler angles ZXY intrinsic vector to Rotation Matrix for Shrimp"""
    r = R.from_euler('ZXY', x)
    return r.as_dcm()

## Relative Wind
Calculating the relative wind is pretty similar to the blade equations above. 
We want to take the given blade section, and then calculate the relative wind vector in the propellor frame. 

![relativewind](../lib/relative_wind.png)

The propellor (E) frame described:

![propellorFrame](../lib/element_frame.png)

Assumptions: No exterior wind disturbance.

The relative wind in this case comes from three components, which we can sum independently:
* **The body velocity**. This is simply the velocity of the body frame w.r.t. the world frame in the world frame $^w v_w^b$ rotated to the element frame $^e v_w^b$. 

$${}^e v_w^b = {}^e R_b {}^b R_w {}^w v_w^b$$

* **The inflow velocity**. The inflow velocity $\nu$ adds simply another component in the negative z-axis, coming from the ducted interaction:

$${}^e v_{inflow} = [0,\ 0,\ -\nu]^T$$

* **The body angular velocity**. The angular velocity in the body frame $\hat{\omega}^b$ needs to be translated to a linear velocity at the element point. The position vector from the body origin (center of mass) to the element origin in the body frame is ${}^b p_b^e$.
${}^b p_b^e = [r \cos\kappa,\ r \sin\kappa,\ h]^T$, where $h$ is the height (z-axis distance) from the cg to the propellor center, $r$ is the radius of the element, and $\kappa$ is the angle of the propellor with respect to the body x axis. 

For calculating the angle of the propellor $\kappa$, we have to take a different approach for body vs. shaft blades.

For the body blades, the first blade is always aligned with the body x-axis and all further blades are spaced evenly around the center of the body. 

The linear velocity of the element relative to the body in the body frame is ${}^b v_e^b = {}^b \omega^b \times {}^b p_b^e$. 

However, if the blades are rotating independently of the body rotation, with some speed $\dot{\psi}$, then there is an extra rotation of the element in the body frame: ${}^b \omega_e = [0,\ 0,\ \dot{\psi}]^T$, such that:

$${}^w v_e^b = {}^w R_b \left(({}^b \omega_b + {}^b \omega_e) \times {}^b p_b^e\right)$$

For the body blade $\dot{\psi}$ is zero, while otherwise it is the propellor spin rate in radians per second.

A note on $\kappa$ for the shaft blade:
For the shaft blades, they are spinning at roughly 3-8 revolutions per second. That is just barely fast enough that we should not track the position of each blade to do the calculation, but instead do the integral. TODO. 

The total relative wind is then:

$${}^e v_{wind} = - {}^e v_w^b - {}^e v_b^e + {}^e v_{inflow}$$

### Calculating rotation matricies
Assuming we know our XYZ intrinsic euler angles, ${}^b R_w$ is simply the transformation from euler to rotation. The rotation from body to element, ${}^e R_b$ is the rotation by the propellor angle. In the picture above, the propellor angle is 0 (the propellor y-axis aligns with the body x-axis), so ${}^e R_b$ is a rotation about the body $z$ axis by $\pi/2$. 

### Calculating angular velocity
The angular velocities we are keeping track of are those of the body frame in the world frame: ${}^w \omega_b = [p\ q\ r]^T$. To get it in the body frame, I am pretty sure you can just rotate it. But to go from the euler angle rotations:

$$\begin{bmatrix} p \\ q \\ r \end{bmatrix} = \begin{bmatrix} \cos \theta & 0 & \cos\phi \sin\theta \\ 0 & 1 & \sin\phi \\ \sin\theta & 0 & \cos\phi \cos\theta \end{bmatrix} 
\begin{bmatrix} \dot{\phi} \\ \dot{\theta} \\ \dot{\psi}\end{bmatrix}$$

In [58]:
def getRelativeWind(bladeIndex, totalNumBlades, bladeRadius, vel_b2w_w, 
                    rot_w2b, angvel_b2w_b, inflowVelocity, height_b2e,
                    prop_angvel):
    """
    Returns relative wind angle as well as the wind angle vector for 
    the given quantities
    
    Return values:
    relativeWind_e : the relative wind in the propellor frame
    relWindAngleGamma: the wind angle relative to the propellor blade disk angle
    r_b2e_b: the vector from the body to the element in the body frame
    rot_b2e: the rotation matrix from the body to element frame 
    
    Arguements:
    bladeIndex: the index of the blade to be used. Assumes blade 0 is aligned with the 
    x-axis of the body
    totalNumBlades: total number of blades, assuming equal spacing
    bladeRadius: the radius to do this overall calculation
    vel_b2w_w: velocity of the body relative to the world in the world frame
    rot_b2w: DCM of body to world 
    angvel_b2w_w: angular velocity of body relative to world in the world frame (p,q,r)
    inflowVelocity: also called nu - scalar amount of inflow velocity
    height_b2e: the distance from body to element along the body z axis 
    prop_angvel: propellor angular velocity (spin rate)
    
    Notation:
    _e is prop element frame
    _b is the body frame (about the center of gravity)
    _w is world/inertial frame
    
    """
    # Calculate Rotation Matricies
    propellor_angle = bladeIndex/totalNumBlades*2*np.pi
    rot_b2e = R.from_euler('Z', propellor_angle + np.pi/2).as_dcm()
    rot_b2w = np.transpose(rot_w2b)
    
    # Body Velocity Calculation
    vel_b2w_b = rot_w2b.dot(vel_b2w_w)
    vel_b2w_e = rot_b2e.dot(vel_b2w_b)
    
        
    # Inflow velocity Calculation
    # it is always in the negative z direction
    vel_inducedInflow_e = np.array([0, 0, -inflowVelocity])
    
    
    # Velocity from rotational component Calculation
    # Calculate the vector from the cg to the element 
    pos_x_blade = bladeRadius*np.cos(propellor_angle + np.pi/2)
    pos_y_blade = bladeRadius*np.sin(propellor_angle + np.pi/2)
    r_b2e_b = np.array([pos_x_blade, pos_y_blade, height_b2e])
    
    # Calculate the angular velocity in the body frame
    angvel_spin_b = np.array([0, 0, prop_angvel])
    # Total angular velocity of the propellor is body + prop
    total_angvel_b = angvel_spin_b + angvel_b2w_b
    
    vel_e2b_b = np.cross(total_angvel_b, r_b2e_b)
    vel_e2b_e = rot_b2e.dot(vel_e2b_b)
    
    # Sum all forces together
    relativeWind_e = -(vel_b2w_e + vel_e2b_e) + vel_inducedInflow_e
    # TODO: confirm this is the arctan we want I guess
    relWindAngleGamma = np.arctan2(relativeWind_e[2], relativeWind_e[0])

    return (relativeWind_e, relWindAngleGamma, r_b2e_b, rot_b2e)

In [61]:
def testRelativeWind():
    
    # Test for relative wind 
    bodyEulerAngles_w = np.pi/180*np.array([0,0,0]) # ZXY
    # Below is technically body2world, but no big deal
    rot_body2world = euler2Rotm(bodyEulerAngles_w) 

    totalNumBlades = 2
    bladeRadius = 0.1
    bladeIndex = 1
    vel_b2w_w = np.array([0, 10, 10])
    angvel_b2w_b = np.array([0.0, 0.0, 0.0])
    inflowVelocity = 0
    height_b2e = 2
    prop_angvel = 0

    # Using XYZ euler angles such that the last angle Z is the correct psi angle
    (relWind, gamma, _, _) = getRelativeWind(bladeIndex, totalNumBlades, bladeRadius, 
                                             vel_b2w_w, rot_body2world, angvel_b2w_b, 
                                             inflowVelocity, height_b2e, prop_angvel)

    print('gamma', gamma*180/np.pi)
    print('relwind', relWind)

testRelativeWind()

gamma -135.0
relwind [-1.00000000e+01  2.22044605e-15 -1.00000000e+01]


## Lift and Drag
Note that lift is POSITIVE in this model. Z is up. 
$$L = \frac{1}{2} \rho V^2 C_L S$$
$$D = \frac{1}{2} \rho V^2 C_D S$$
Where 
* $C_L, C_D$ are lift and drag coefficients, functions of angle of attack ($alpha$)
* $S$ is wing area. 
* $\rho$ is air density 
* $V$ is velocity at the wing element

In [62]:
def getAngleOfAttack(bladePitch, relWindAngleGamma):
    """ Returns angle of attack between -pi and pi"""
    unwrapped_angle = bladePitch + relWindAngleGamma
    return (unwrapped_angle + np.pi) % (2 * np.pi ) - np.pi

def clFlatPlate(alpha):
    """ Coeff of lift for a Flat Plate """
    return 2.*np.sin(alpha)*np.cos(alpha)


def cdFlatPlate(alpha):
    """ Coeff of drag for a Flat Plate """
    return 2.*np.sin(alpha)**2.0


def cmFlatPlate(alpha):
    """ Coeff of pitch moment for a Flat Plate """
    return 0.0

def getLiftDragElement(elementWidth, chord, rho, relWind_e, alpha):
    """ Returns magnitude of lift and drag
    Arguements:
    elementWidth: the width of the blade section 
    chord: airfoil chord
    rho: air density
    relWind_e : relative wind in the propellor frame
    alpha: angle of attack of section
    Returns:
    (lift, drag) scalars
    """
    # velocity only incorporates the x and z components in the wind frame 
    speed = relWind_e.dot([1,0,1])    
    coeffLift = clFlatPlate(alpha)
    coeffDrag = cdFlatPlate(alpha)
    quantity = 0.5*speed*speed*chord*elementWidth*rho
    lift = quantity * coeffLift
    drag = quantity * coeffDrag
    return (lift, drag)


In [33]:
def rpm2RadiansPerSecond(x:float) -> float:
     return 0.10472 * x

In [34]:
class BladeElementParameters:
    def __init__(self, rho, pitch, width, radius, chord, index, totalNumBlades, 
                 height_body2element, inflowVel, spinVel):
        self.rho = rho
        self.pitch = pitch 
        self.width = width  # this is in the radial direction (strip model width)
        self.radius = radius  # this should probably "middle" of the overall radius width 
        self.chord = chord
        self.index = index # Blade Index out of n blades
        self.totalNumBlades = totalNumBlades # total number of blades. A normal prop has 2
        self.height_body2element = height_body2element # scalar [m]
        self.inflowVel = inflowVel # Inflow velocity for this element. Scalar [m/s]
        self.spinVel = spinVel # spin velocity for this element. Scalar [m/s]

class ShrimpState:
    """ Shrimp state object. Stores state in a clear format, and also converts 
        the clear format to a numpy vector for ODE integration.
    """
    def __init__(self, r_b2w_w, vel_b2w_w, euler_b2w_w, angvel_b2w_w, inflowVel, prop_angVel):
        self.r_b2w_w = r_b2w_w  # np.array 3x1 position vector from world origin to body/cg
        self.vel_b2w_w = vel_b2w_w  # np.array 3x1 velocity vector from world origin to body/cg
        self.euler_b2w_w = euler_b2w_w  # np.array 3x1 euler angles vector from world to body frame
        self.angvel_b2w_w = angvel_b2w_w  # np.array 3x1 euler velocity vector from world to body frame
        self.inflowVel = inflowVel # scalar inflow velocity adjustment due to duct on the shaft propellor [m/s]
        self.prop_angVel = prop_angVel # the speed of the spin of the propellor. Scalar [rad/s]

    def toStateVec(self):
        raise Exception('TODO: implement')
    
    
# Blade equations 
def getElementForceMoment(bladeParams, state):
    """Returns force and moment vectors in the flyer frame 
    Assumes the element radius is the radius at the center of the width. 
    The width is only used in a surface area calculation. 
    Arguments:
        bladeParams (BladeElementParameters)
        state (ShrimpState)
    """
    
    rho = bladeParams.rho
    bladeIndex = bladeParams.index
    bladeRadius = bladeParams.radius
    vel_b2w_w = state.vel_b2w_w
    angvel_b2w_w = state.angvel_b2w_w
    inflowVelocity = bladeParams.inflowVel
    height_body2element = bladeParams.height_body2element
    prop_angvel = bladeParams.spinVel
    euler_b2w = state.euler_b2w_w
    totalNumBlades = bladeParams.totalNumBlades
    
    rot_body2world = euler2Rotm(euler_b2w)
    # TODO: should be rot_flyer2world, need to iron this out 
    # more explicitly and write it up
    (relativeWind_e, gamma, r_b2Element_f, rot_flyer2prop) = getRelativeWind(bladeIndex, totalNumBlades, 
                                                                              bladeRadius, vel_b2w_w, 
                                                                              rot_body2world, angvel_b2w_w, 
                                                                              inflowVelocity, height_body2element,
                                                                              prop_angvel)
    
    alpha = getAngleOfAttack(gamma, bladeParams.pitch)
    (lift, drag) = getLiftDragElement(bladeParams.width, bladeParams.chord, rho, relativeWind_e, alpha)
    
    # Forces in the propellor frame
    normalForce_e = lift*np.cos(gamma) + drag*np.sin(gamma)
    tangentialForce_e = -lift*np.sin(gamma) + drag*np.cos(gamma)
    
    # Z-axis is up! 
    forces_e = np.array([-tangentialForce_e, 0, normalForce_e])
    
    rot_prop2flyer = np.transpose(rot_flyer2prop)
    forces_f = rot_prop2flyer.dot(forces_e)
    moments_f = np.cross(forces_f, r_b2Element_f)
    print('Element forces', forces_f)
    return(forces_f, moments_f)
    
def testBladeElement(radius, chord, pitch, width):
    rho = 1.225
    bladeIndex = 1
    totalNumBlades = 2
    height_body2element = 0.01
    
    inflowVel = 1
    prop_angVel = rpm2RadiansPerSecond(500)
    # Set up this particular propellor element
    bladeParams = BladeElementParameters(rho, pitch, width, radius, chord, bladeIndex,
                                          totalNumBlades, height_body2element, inflowVel, prop_angVel)
    
    r_b2w_w = np.array([0.1, 0.1, 0.1])
    vel_b2w_w = np.array([0.1, 0, 0])
    euler_b2w_w = np.array([10, 0, 40]) * np.pi/180.
    angvel_b2w_w = np.array([0.01, 0, 0.02])
    inflowVel = 1
    prop_angVel = rpm2RadiansPerSecond(500)
    state = ShrimpState(r_b2w_w, vel_b2w_w, euler_b2w_w, angvel_b2w_w, inflowVel, prop_angVel)
    
    (forces_f, moments_f) = getElementForceMoment(bladeParams, state) 
    return (forces_f, moments_f)
    
def testBlade():
    n = 5 # The number of sections to split the blade into for element theory
    radiusRoot = 0
    radiusTip = 0.1 # maximum blade radius / tip to root
    width = (radiusTip-radiusRoot)/n
    chordRoot = 0.1 # Chord at the root of the blade
    chordTip = 0.2 # Chord at the tip of the blade
    pitchRoot = 0*np.pi/180
    pitchTip = 15*np.pi/180
    
    radii = np.linspace(radiusRoot, radiusTip, n)
    chords = np.linspace(chordRoot, chordTip, n)
    pitches = np.linspace(pitchRoot, pitchTip, n)
    
    forceMomentTuples = [testBladeElement(r, c, p, width) for (r,c,p) in zip(radii, chords, pitches)]

    forces = sum([f for f,_ in forceMomentTuples])
    moments = sum([m for _,m in forceMomentTuples])
    print('Forces', forces)
    print('Moments', moments)
testBlade()

Element forces [ 2.07636762e-20 -1.74227931e-20 -1.97466460e-03]
Element forces [-1.34974527e-05  1.13257076e-05 -2.68824264e-04]
Element forces [-0.00023855  0.00020016 -0.00236532]
Element forces [-0.00027502  0.00023077 -0.00180486]
Element forces [ 0.00140312 -0.00117736  0.0068358 ]
Forces [ 0.00087606 -0.0007351   0.00042213]
Moments [ 0.00031686 -0.00028081 -0.00014023]


In [ ]:
def renderShrimp(states, times):
    """ Makes a 3D rendering of ShrimpStates at given times
        Argumenets:
            states: a list of shrimp states corresponding to given times
            times: a np.ndarray of times to plot at
    """
    
    # Allen Code here 